### ACT Crimes

This script will import the original ACT crime statistics file 'Website Stats Monthly Download.xlsx' which is published monthly by the ACT police on the following website:  
https://www.policenews.act.gov.au/crime-statistics-and-data/crime-statistics

The file will betransformed to be suitable for the Power BI project and will be merged with the district population taken from the 2021 census.

**Import the required libraries**

In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
from datetime import datetime

**Load the Monthly Stats XLSX file and create the data frames**

In [2]:
# Path to the monthly stats file Excel file
file_path = 'Website Stats Monthly Download.xlsx'

# Load the Excel file
with pd.ExcelFile(file_path, engine='openpyxl') as xls:
    # Read the first sheet "Offence Statistics" into the data frame df_dist without a header 
    df_dist = pd.read_excel(xls, sheet_name=xls.sheet_names[0], header=None)
    
    # Read the second sheet "Traffic Statistics - ACT" into the data frame df_traf without a header 
    df_traf = pd.read_excel(xls, sheet_name=xls.sheet_names[1], header=None)
    
    # Read the third sheet "Family Violence Statistics - ACT" into the data frame df_fv without a header 
    df_fv = pd.read_excel(xls, sheet_name=xls.sheet_names[2], header=None)


In [3]:
# Display the first few rows of the data frame
df_traf.head(20)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,Table 11: ACT - Traffic and Transport Crime St...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PROMIS and Autocite as at 1 September 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,Traffic - Issue Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,AUG24,JUL24,JUN24,MAY24,APR24,MAR24,FEB24,JAN24,DEC23,...,OCT14,SEP14,AUG14,JUL14,JUN14,MAY14,APR14,MAR14,FEB14,JAN14
5,Total Traffic Infringement Notices (TINs) (inc...,592,481,372,591,568,355,521,580,366,...,1072,1194,1563,1450,1064,1251,1174,1055,975,1230
6,Total Traffic Infringement cautions (including...,267,238,226,267,243,164,267,263,199,...,601,576,772,770,480,678,681,557,588,564
7,Speeding TINs,193,164,118,279,234,134,174,193,161,...,257,403,579,568,331,431,349,303,353,361
8,Speeding cautions,43,69,40,71,46,43,52,67,44,...,144,229,295,282,150,223,202,160,182,156
9,Number of persons charged with drink driving o...,34,37,57,60,50,44,44,66,68,...,81,103,154,89,102,135,83,107,73,90


In [4]:
df_dist.shape

(199, 129)

**Create a function to transform the data frames to be suitable for the Power BI project**

In [5]:
def prepare_df(df):
    
    # Find the first row where the value in column 2 is not NaN
    head_row = df[2].first_valid_index()
    
    # Set this row to be the new header for the data frame
    new_header = df.iloc[head_row]
    # print(new_header)
    
    # Rename the columns based on the new header
    df.columns = new_header
    
    # Keep all rows including the original ones
    df = df.reset_index(drop=True)
    
    # Create a new column and place the date columns into a list
    new_columns = ['Offence']
    month_year_columns = df.columns[1:]

    # Rename the date columns to the last day of the month
    for col in month_year_columns:
        # Convert month-year string to datetime object
        date_obj = datetime.strptime(col, '%b%y')  
        # Get the last day of the month
        next_month = date_obj.replace(day=28) + pd.DateOffset(days=4)
        last_day = next_month - pd.DateOffset(days=next_month.day)
        new_columns.append(last_day.strftime('%d/%m/%Y'))
        
    # Rename the columns based on the new date names
    df.columns = new_columns
    
    # Convert the "Offence" column to string for proper filtering
    df['Offence'] = df['Offence'].astype(str)

    # Filter out unwanted rows where 'Offence' is NaN, blank, 'Total', 'Offence', 'Offences', or starts with 'PROMIS'
    df = df[
        (df['Offence'].notna()) & 
        (df['Offence'].str.strip() != '') &
        (df['Offence'] != 'nan') & 
        (df['Offence'] != 'Total') & 
        (df['Offence'] != 'Offence') & 
        (df['Offence'] != 'Offences') &
        (~df['Offence'].str.startswith('PROMIS'))
        ]

    # Create the "District" column and initialize with empty strings
    df.insert(1, 'District', '')
    
    # Fill the "District" column based on the pattern in the "Offence" column
    current_district = ""
    for i, row in df.iterrows():
        if row['Offence'].startswith('Table'):
            # Extract the district name from the "Offence" column
            current_district = row['Offence'].split(': ')[1].split(' -')[0]
            print(f'District: {current_district}')
            # Rename certain districts
            if current_district == "Weston":
                current_district = "Weston Creek"
            elif current_district == "Molonglo District":
                current_district = "Molonglo"
            elif current_district == "Other Areas":
                current_district = "Rural"
        # Apply the district name to the District column
        df.at[i, 'District'] = current_district

    # Delete all rows where 'Offence' starts with 'Table'
    df = df[~df['Offence'].str.startswith('Table')]  
    
    # Reset the index of the data frame after filtering
    df = df.reset_index(drop=True)
    
    return(df)

**Prepare the "Offence Statistics by District" tab**

In [6]:
df_dist.head(20)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,Table 1: ACT - Offences and other activities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"PROMIS, Autocite and ACT Policing Fatal Regist...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Offence,Date reported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,AUG24,JUL24,JUN24,MAY24,APR24,MAR24,FEB24,JAN24,DEC23,...,OCT14,SEP14,AUG14,JUL14,JUN14,MAY14,APR14,MAR14,FEB14,JAN14
5,Assault,227,257,210,234,203,234,278,259,283,...,146,132,177,160,166,163,157,198,157,154
6,Burglary,94,111,117,156,113,163,159,160,144,...,222,183,193,174,209,221,162,231,151,160
7,Homicide,0,0,2,2,0,2,0,1,1,...,1,0,0,1,0,0,0,0,1,1
8,Motor vehicle theft,61,64,57,79,80,105,106,100,87,...,103,54,64,63,50,74,68,64,50,90
9,Offences against a person,32,37,28,35,23,23,25,26,30,...,6,10,11,10,3,9,12,11,7,3


In [7]:
# Call the function to transform the data drame for df_district
df_dist = prepare_df(df_dist)

District: ACT
District: Belconnen
District: Gungahlin
District: Inner North
District: Inner South
District: Weston
District: Molonglo District
District: Woden
District: Tuggeranong
District: Other Areas


In [8]:
# Create a new data frame for rows where District is "ACT"
df_act = df_dist[df_dist['District'] == 'ACT']

# Remove rows where District is "ACT" from df_dist
df_dist = df_dist[df_dist['District'] != 'ACT']

# Reset the index of the DataFrame after removing the rows
df_dist = df_dist.reset_index(drop=True)

In [9]:
# View the header to ensure the integrity
df_dist.head(26)

,Offence,District,31/08/2024,31/07/2024,30/06/2024,31/05/2024,30/04/2024,31/03/2024,29/02/2024,31/01/2024,...,31/10/2014,30/09/2014,31/08/2014,31/07/2014,30/06/2014,31/05/2014,30/04/2014,31/03/2014,28/02/2014,31/01/2014
0,Assault,Belconnen,49,72,41,52,47,58,47,60,...,31,27,42,35,27,32,32,42,35,26
1,Burglary,Belconnen,15,15,28,26,15,31,36,27,...,79,54,37,33,33,39,30,41,37,43
2,Homicide,Belconnen,0,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,0,1,0
3,Motor vehicle theft,Belconnen,20,8,22,16,13,17,24,22,...,22,17,5,14,9,20,16,20,16,24
4,Offences against a person,Belconnen,7,7,5,3,1,7,2,6,...,0,1,2,1,0,1,1,3,2,2
5,Other offences,Belconnen,118,151,130,136,129,133,117,167,...,171,149,177,228,174,187,183,216,252,169
6,Property damage,Belconnen,42,48,57,60,45,47,72,76,...,59,56,63,56,55,58,73,72,52,92
7,Road Collision with injury,Belconnen,8,5,4,6,4,7,4,7,...,13,11,10,14,12,15,10,17,12,7
8,Road Fatality,Belconnen,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
9,Robbery,Belconnen,1,3,1,3,1,1,3,3,...,7,2,1,5,3,1,2,1,1,5


In [10]:
# View df_act to ensure the integrity
df_act

,Offence,District,31/08/2024,31/07/2024,30/06/2024,31/05/2024,30/04/2024,31/03/2024,29/02/2024,31/01/2024,...,31/10/2014,30/09/2014,31/08/2014,31/07/2014,30/06/2014,31/05/2014,30/04/2014,31/03/2014,28/02/2014,31/01/2014
0,Assault,ACT,227,257,210,234,203,234,278,259,...,146,132,177,160,166,163,157,198,157,154
1,Burglary,ACT,94,111,117,156,113,163,159,160,...,222,183,193,174,209,221,162,231,151,160
2,Homicide,ACT,0,0,2,2,0,2,0,1,...,1,0,0,1,0,0,0,0,1,1
3,Motor vehicle theft,ACT,61,64,57,79,80,105,106,100,...,103,54,64,63,50,74,68,64,50,90
4,Offences against a person,ACT,32,37,28,35,23,23,25,26,...,6,10,11,10,3,9,12,11,7,3
5,Other offences,ACT,548,594,563,609,555,602,584,747,...,750,798,1126,841,802,812,861,842,801,1168
6,Property damage,ACT,241,237,236,215,230,244,267,300,...,376,330,350,327,303,338,355,353,288,429
7,Road Collision with injury,ACT,25,23,26,31,19,32,25,21,...,47,48,60,55,57,64,51,70,43,53
8,Road Fatality,ACT,0,0,1,1,1,1,0,1,...,0,2,0,0,1,0,0,3,2,0
9,Robbery,ACT,8,7,10,12,13,14,15,16,...,18,7,11,21,17,9,8,17,12,17


**Prepare the "Traffic Statistics - ACT" tab**

In [11]:
# View the header before the transformation
df_traf.head(20)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,Table 11: ACT - Traffic and Transport Crime St...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PROMIS and Autocite as at 1 September 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,Traffic - Issue Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,AUG24,JUL24,JUN24,MAY24,APR24,MAR24,FEB24,JAN24,DEC23,...,OCT14,SEP14,AUG14,JUL14,JUN14,MAY14,APR14,MAR14,FEB14,JAN14
5,Total Traffic Infringement Notices (TINs) (inc...,592,481,372,591,568,355,521,580,366,...,1072,1194,1563,1450,1064,1251,1174,1055,975,1230
6,Total Traffic Infringement cautions (including...,267,238,226,267,243,164,267,263,199,...,601,576,772,770,480,678,681,557,588,564
7,Speeding TINs,193,164,118,279,234,134,174,193,161,...,257,403,579,568,331,431,349,303,353,361
8,Speeding cautions,43,69,40,71,46,43,52,67,44,...,144,229,295,282,150,223,202,160,182,156
9,Number of persons charged with drink driving o...,34,37,57,60,50,44,44,66,68,...,81,103,154,89,102,135,83,107,73,90


In [12]:
# Call the function to transform the data drame for df_traffic
df_traf = prepare_df(df_traf)

District: ACT


In [13]:
# View the data frame after the transformation
df_traf

,Offence,District,31/08/2024,31/07/2024,30/06/2024,31/05/2024,30/04/2024,31/03/2024,29/02/2024,31/01/2024,...,31/10/2014,30/09/2014,31/08/2014,31/07/2014,30/06/2014,31/05/2014,30/04/2014,31/03/2014,28/02/2014,31/01/2014
0,Total Traffic Infringement Notices (TINs) (inc...,ACT,592,481,372,591,568,355,521,580,...,1072,1194,1563,1450,1064,1251,1174,1055,975,1230
1,Total Traffic Infringement cautions (including...,ACT,267,238,226,267,243,164,267,263,...,601,576,772,770,480,678,681,557,588,564
2,Speeding TINs,ACT,193,164,118,279,234,134,174,193,...,257,403,579,568,331,431,349,303,353,361
3,Speeding cautions,ACT,43,69,40,71,46,43,52,67,...,144,229,295,282,150,223,202,160,182,156
4,Number of persons charged with drink driving o...,ACT,34,37,57,60,50,44,44,66,...,81,103,154,89,102,135,83,107,73,90
5,Person charged with unlicenced driving offence...,ACT,50,31,34,43,39,56,32,75,...,68,78,92,89,65,103,66,81,64,87
6,Person charged with unlicenced driving offence...,ACT,9,4,9,9,6,4,9,21,...,7,8,9,14,6,8,11,11,4,14
7,Road crashes resulting in injury (not fatalities),ACT,25,23,26,31,19,32,25,21,...,47,48,60,55,57,64,51,70,43,53
8,Road crashes resulting in death,ACT,0,0,1,1,1,1,0,1,...,0,2,0,0,1,0,0,3,2,0


In [14]:
# Call the function to transform the data drame for df_family_violence
df_fv = prepare_df(df_fv)

District: ACT


In [15]:
# View the data frame after the transformation
df_fv

,Offence,District,31/08/2024,31/07/2024,30/06/2024,31/05/2024,30/04/2024,31/03/2024,29/02/2024,31/01/2024,...,31/10/2014,30/09/2014,31/08/2014,31/07/2014,30/06/2014,31/05/2014,30/04/2014,31/03/2014,28/02/2014,31/01/2014
0,Family violence related incidents,ACT,309,342,313,322,318,344,337,376,...,232,228,226,239,267,280,238,296,306,284
1,Family violence related offences,ACT,219,240,231,214,201,252,241,249,...,135,101,118,125,101,144,121,122,105,111
2,Family violence related assault offences,ACT,76,81,68,65,64,81,89,97,...,66,35,58,56,44,63,47,55,44,48
3,Number of offenders apprehended where family v...,ACT,67,77,53,73,63,74,70,60,...,27,26,35,32,26,23,31,32,26,26


In [16]:
# Concatenate df_traf and df_fv with df_act
df_act = pd.concat([df_act, df_traf, df_fv], ignore_index=True)

**Change and remove certain columns from the combined df_act**

Some rows were duplicated in the different sheets in the monthly dataset. Duplicates will be removed.  
Other offence names will be slightly modified.

In [17]:
# Map the original names to new names
name_changes = {
    "Assault": "Assault",
    "Burglary": "Burglary",
    "Homicide": "Homicide",
    "Motor vehicle theft": "Motor vehicle theft",
    "Offences against a person": "Offences against a person",
    "Other offences": "Other offences",
    "Property damage": "Property damage",
    "Road Collision with injury": "Road Collision with injury",
    "Road Fatality": "Road Fatality",
    "Robbery": "Robbery",
    "Sexual Assault": "Sexual Assault",
    "Theft (excluding motor vehicles)": "Theft (excluding motor vehicles)",
    "Traffic infringement notices": "Traffic infringement notices - All",
    "Total Traffic Infringement Notices (TINs) (including speeding)": None,  # Removed
    "Total Traffic Infringement cautions (including speeding)": "Total Traffic Infringement cautions - All",
    "Speeding TINs": "Speeding Infringements",
    "Speeding cautions": "Speeding cautions",
    "Number of persons charged with drink driving offences": "Drink driving offences",
    "Person charged with unlicenced driving offences - licence disqualified or suspended": "Unlicenced Driving - disqualified or suspended licence",
    "Person charged with unlicenced driving offences - licence never held": "Unlicenced driving - licence never held",
    "Road crashes resulting in injury (not fatalities)": None,  # Removed
    "Road crashes resulting in death": None,  # Removed
    "Family violence related incidents": "Family violence: incidents",
    "Family violence related offences": "Family violence: offences",
    "Family violence related assault offences": "Family violence: assault offences",
    "Number of offenders apprehended where family violence related": "Family violence: offenders apprehended"
}

# Filter the data frame to keep only rows with offence names present in the dictionary keys
df_act = df_act[df_act['Offence'].isin(name_changes.keys())]

# Map the offence names to their new values using .loc
df_act.loc[:, 'Offence'] = df_act['Offence'].map(name_changes)

# Remove duplicate rows
df_act = df_act[df_act['Offence'].notna()]

In [18]:
# Reset the index of the data frame after removing some rows
df_act = df_act.reset_index(drop=True)

In [19]:
# View the final data frame
df_act

,Offence,District,31/08/2024,31/07/2024,30/06/2024,31/05/2024,30/04/2024,31/03/2024,29/02/2024,31/01/2024,...,31/10/2014,30/09/2014,31/08/2014,31/07/2014,30/06/2014,31/05/2014,30/04/2014,31/03/2014,28/02/2014,31/01/2014
0,Assault,ACT,227,257,210,234,203,234,278,259,...,146,132,177,160,166,163,157,198,157,154
1,Burglary,ACT,94,111,117,156,113,163,159,160,...,222,183,193,174,209,221,162,231,151,160
2,Homicide,ACT,0,0,2,2,0,2,0,1,...,1,0,0,1,0,0,0,0,1,1
3,Motor vehicle theft,ACT,61,64,57,79,80,105,106,100,...,103,54,64,63,50,74,68,64,50,90
4,Offences against a person,ACT,32,37,28,35,23,23,25,26,...,6,10,11,10,3,9,12,11,7,3
5,Other offences,ACT,548,594,563,609,555,602,584,747,...,750,798,1126,841,802,812,861,842,801,1168
6,Property damage,ACT,241,237,236,215,230,244,267,300,...,376,330,350,327,303,338,355,353,288,429
7,Road Collision with injury,ACT,25,23,26,31,19,32,25,21,...,47,48,60,55,57,64,51,70,43,53
8,Road Fatality,ACT,0,0,1,1,1,1,0,1,...,0,2,0,0,1,0,0,3,2,0
9,Robbery,ACT,8,7,10,12,13,14,15,16,...,18,7,11,21,17,9,8,17,12,17


**Load the District Population XLSX file and create the population data frame**

In [20]:
# Path to the population Excel file
file_path = 'District Population.xlsx'

# Load the Excel file
with pd.ExcelFile(file_path, engine='openpyxl') as xls:
    # Read the sheet into the data frame df_pop with a header
    df_pop = pd.read_excel(xls, sheet_name=xls.sheet_names[0])

In [21]:
# Display the population by district data frame.
df_pop

,District,Population
0,Belconnen,106061
1,Gungahlin,87682
2,Inner North,61188
3,Inner South,31592
4,Weston Creek,24630
5,Molonglo,11435
6,Woden Valley,39279
7,Tuggeranong,89461
8,Rural,3171
9,ACT,454499


**Create the wrangled XLSX file to be used in the Power BI Dashboard**

In [22]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
output_file_path = 'Website Stats Monthly Wrangled.xlsx'
writer = pd.ExcelWriter(output_file_path, engine='xlsxwriter')

# Write each data frame to a specific sheet.
df_dist.to_excel(writer, sheet_name='Offence Statistics by District', index=False)
df_act.to_excel(writer, sheet_name='Offence Statistics - ACT', index=False)
df_pop.to_excel(writer, sheet_name='Population', index=False)

# Save the Excel file
writer.close()

print(f"Data successfully written to: {output_file_path}")

Data successfully written to: Website Stats Monthly Wrangled.xlsx
